In [13]:
from requests import get, post

url_root = "http://127.0.0.1:8000"
get(f"{url_root}/handshake")

<Response [200]>

In [14]:
res = get(f"{url_root}/interactions/list/?limit=1")

res.json()

{'interaction_entries': [['511145.b0001',
   {'root': '511145.b0001',
    'data': [{'target': '511145.b0810',
      'ppid': '23a44cc647a34c18a32541e786080227'}]},
   [{'ppid': '23a44cc647a34c18a32541e786080227',
     'protein1': '511145.b0001',
     'protein2': '511145.b0810',
     'neighborhood': 0,
     'fusion': 0,
     'cooccurence': 0,
     'coexpression': 0,
     'experimental': 0,
     'database': 0,
     'textmining': 162,
     'combined_score': 161}]]]}

In [21]:
uniprot_grp = ["P0AD86", "P0AEQ6"]

res = post(f"{url_root}/interactions/matrix/", json= { "uniprotIDs": uniprot_grp } )

res.json()

{'members': ['P0AD86', 'P0AEQ6'],
 'indexed_scores': [[1,
   {'ppid': '23a44cc647a34c18a32541e786080227',
    'protein1': '511145.b0001',
    'protein2': '511145.b0810',
    'neighborhood': 0,
    'fusion': 0,
    'cooccurence': 0,
    'coexpression': 0,
    'experimental': 0,
    'database': 0,
    'textmining': 162,
    'combined_score': 161}]]}

In [1]:
%load_ext autoreload
%autoreload 2
from requests import post
import sys
sys.path.append("/Users/guillaumelaunay/work/projects/service/quantitativeProteomics/pyT2GA/pyrediscore")  
sys.path.append("/Users/guillaumelaunay/work/projects/service/quantitativeProteomics/pyT2GA/pystringbio")
%autoreload 2

In [2]:
from pystringbio.io.mapping import parseMappingRulesFile
from pystringbio.api import schemas

In [3]:
from pydantic import validate_arguments
from typing import List
from pystringbio.api.schemas import UniprotAC


In [5]:
from pystringbio.store import PyStringBioStore
alias_file       = "/Users/guillaumelaunay/work/projects/service/quantitativeProteomics/data/ppi/string/511145.protein.aliases.v11.5.txt"
interaction_file = "/Users/guillaumelaunay/work/projects/service/quantitativeProteomics/data/ppi/string/511145.protein.links.detailed.v11.5.HEAD1.txt"
pystring_store = PyStringBioStore()
pystring_store.wipe()
with open(alias_file) as f1:
    pystring_store.load_aliases_stream(f1)
with open(interaction_file) as f2:
    pystring_store.load_interactions_stream(f2)

for p in pystring_store.interactions:
    print(p)
    
    
    

Nothing to wipe under InteractionDatum:*
[b'{"ppid": "23a44cc647a34c18a32541e786080227", "protein1": "511145.b0001", "protein2": "511145.b0810", "neighborhood": 0, "fusion": 0, "cooccurence": 0, "coexpression": 0, "experimental": 0, "database": 0, "textmining": 162, "combined_score": 161}']
('511145.b0810', RefStringPpiTree(root='511145.b0810', data=[RefStringPpiNode(target='511145.b0001', ppid='23a44cc647a34c18a32541e786080227')]), [InteractionDatum(ppid='23a44cc647a34c18a32541e786080227', protein1='511145.b0001', protein2='511145.b0810', neighborhood=0, fusion=0, cooccurence=0, coexpression=0, experimental=0, database=0, textmining=162, combined_score=161)])
[b'{"ppid": "23a44cc647a34c18a32541e786080227", "protein1": "511145.b0001", "protein2": "511145.b0810", "neighborhood": 0, "fusion": 0, "cooccurence": 0, "coexpression": 0, "experimental": 0, "database": 0, "textmining": 162, "combined_score": 161}']
('511145.b0001', RefStringPpiTree(root='511145.b0001', data=[RefStringPpiNode(ta

In [23]:
#uniprot_grp = ["P75860", "P0AD86", "P75616", "P37005", "P00561"]
uniprot_grp = ["P0AD86", "P0AEQ6"]
pystring_store.get_interaction_matrix(uniprot_grp)

InteractionMatrix(members=['P0AD86', 'P0AEQ6'], indexed_scores=[(1, InteractionDatum(ppid='23a44cc647a34c18a32541e786080227', protein1='511145.b0001', protein2='511145.b0810', neighborhood=0, fusion=0, cooccurence=0, coexpression=0, experimental=0, database=0, textmining=162, combined_score=161))])

In [20]:
import re

slice_expre = re.compile(
    r'({})'
)

[4, 5, 6]

In [25]:
# Parsing string.aliases file
class StringIOError(Exception):
    pass

from pystringbio.api.schemas import UniprotToString, StringToUniprot, InteractionDatum
from typing import List, Tuple, Optional
from io import StringIO


def load_mapper_from_stream(stream:StringIO, uniprotKB_only=True)->Optional[ Tuple[ List[UniprotToString], List[StringToUniprot] ] ]:
        buff_string_to_uniprot = {}
        buff_uniprot_to_string = {}
        for l in stream:
            l = l.rstrip('\n')
            if l.startswith("#"):
                continue
            _ = l.split('\t')
            if len(_) != 3:
                raise StringIOError(f"Unexpected number of fields[{len(_)}] on :{l}")
            (string_protein_id, alias, source) = _;
            uniprot_id=None
            if alias.startswith('UniProtKB/Swiss-Prot:'):
                uniprot_id = alias.split(':')[1]

            if not uniprotKB_only and source == 'BLAST_UniProt_AC':
                uniprot_id = alias
                
            if uniprot_id is None:
                continue
                
            if not string_protein_id in buff_string_to_uniprot:
                buff_string_to_uniprot[string_protein_id] = set()
            buff_string_to_uniprot[string_protein_id].add(uniprot_id)
            if not uniprot_id in buff_uniprot_to_string:
                buff_uniprot_to_string[uniprot_id] = set()
            buff_uniprot_to_string[uniprot_id].add(string_protein_id)
                
    
        _a= store.madd( [\
                         UniprotToString(fr=u, to=list(s_set)) \
                         for u, s_set in buff_uniprot_to_string.items()\
                        ])
        _b = store.madd( [\
                          StringToUniprot( fr=s, to=list(u_set) ) \
                          for s, u_set in buff_string_to_uniprot.items()\
                         ])
        return (_a, _b)
#print(_)
#print(t[1][0])
#store.madd(t[1][:])            
#    return \
#        [ UniprotToString(fr=u, to=list(s_set))\
#         for u, s_set in buff_uniprot_to_string.items() ],\
#        [ StringToUniprot( fr=s, to=list(u_set) )  for s, u_set in buff_string_to_uniprot.items() ]
    
path = "/Users/guillaumelaunay/work/projects/service/quantitativeProteomics/data/ppi/string/511145.protein.aliases.v11.5.txt"
with open(path, 'r') as f:
    t = load_mapper_from_stream(f,
                             uniprotKB_only = True)
    print(t)    


NameError: name 'store' is not defined

In [4]:
# Redis value injection of mapper
sys.path.append("/Users/guillaumelaunay/work/projects/service/quantitativeProteomics/pyT2GA/pyrediscore")
from pyrediscore.redantic import RedisStore
store = RedisStore("127.0.0.1", 6379)
from pydantic import BaseModel
from datetime import datetime
from typing import List, Optional

store.load_model(UniprotToString, 'fr')
store.load_model(StringToUniprot, 'fr')

In [5]:
print(t[0][0])
_ = store.madd(t[0][:])
print(_)
print(t[1][0])
store.madd(t[1][:])

fr='P0AD86' to=['511145.b0001']
4147
fr='511145.b0001' to=['P0AD86']


4109

In [9]:
_ = store.get('P0AD86', UniprotToString)
print(_)
store.delete('P0AD86', UniprotToString)

[ k for k in store.list_key(model=UniprotToString) ]

fr='P0AD86' to=['511145.b0001']
UniprotToString:*


['UniprotToString:P00547', 'UniprotToString:P00561']

In [8]:
store.wipe()

3

### Adding interaction 
- Parsing
- Injecting

In [71]:

from pystringbio.io.interactions import parse_interactions_from_stream
with open(i_file) as f:
    d = parse_interactions_from_stream(f)
    print(d[:5])

[InteractionDatum(ppid='ab5ca0f6820641b6967696dd391d16fd', protein1='511145.b0001', protein2='511145.b0810', neighborhood=0, fusion=0, cooccurence=0, coexpression=0, experimental=0, database=0, textmining=162, combined_score=161), InteractionDatum(ppid='bb8703b256fb4745ba500781ec9f3fa6', protein1='511145.b0001', protein2='511145.b0946', neighborhood=0, fusion=0, cooccurence=0, coexpression=0, experimental=0, database=0, textmining=200, combined_score=200), InteractionDatum(ppid='68260b89eb5b4cbf84b98ed4896965a4', protein1='511145.b0001', protein2='511145.b4403', neighborhood=0, fusion=0, cooccurence=0, coexpression=0, experimental=0, database=0, textmining=363, combined_score=363), InteractionDatum(ppid='3ffd34b51b7c43f391525a0599548a36', protein1='511145.b0001', protein2='511145.b2126', neighborhood=0, fusion=0, cooccurence=0, coexpression=0, experimental=0, database=0, textmining=231, combined_score=230), InteractionDatum(ppid='26289ed700ab430e95759e900c32ea1b', protein1='511145.b000

In [6]:
from pystringbio.io.interactions import parseInteractionFile
i_file="/Users/guillaumelaunay/work/projects/service/quantitativeProteomics/data/ppi/string/511145.protein.links.detailed.v11.5.HEAD50.txt"
i_data = parseInteractionFile(i_file)
i_data[0].dict()




Parsing interaction data, this may take a while...


{'ppid': '11d757a86ff64252a59c1e95e5e623e0',
 'protein1': '511145.b0001',
 'protein2': '511145.b0810',
 'neighborhood': 0,
 'fusion': 0,
 'cooccurence': 0,
 'coexpression': 0,
 'experimental': 0,
 'database': 0,
 'textmining': 162,
 'combined_score': 161}

In [7]:
i_data[:5]

[InteractionDatum(ppid='11d757a86ff64252a59c1e95e5e623e0', protein1='511145.b0001', protein2='511145.b0810', neighborhood=0, fusion=0, cooccurence=0, coexpression=0, experimental=0, database=0, textmining=162, combined_score=161),
 InteractionDatum(ppid='c403f3511d3d406cba0ff842df850088', protein1='511145.b0001', protein2='511145.b0946', neighborhood=0, fusion=0, cooccurence=0, coexpression=0, experimental=0, database=0, textmining=200, combined_score=200),
 InteractionDatum(ppid='59a4c8635e6a4c84b2b331847586585a', protein1='511145.b0001', protein2='511145.b4403', neighborhood=0, fusion=0, cooccurence=0, coexpression=0, experimental=0, database=0, textmining=363, combined_score=363),
 InteractionDatum(ppid='87068a63957d490285b132d0cdb0304c', protein1='511145.b0001', protein2='511145.b2126', neighborhood=0, fusion=0, cooccurence=0, coexpression=0, experimental=0, database=0, textmining=231, combined_score=230),
 InteractionDatum(ppid='29f753fc9b9e457b9954efcd6424eeb4', protein1='511145.

In [54]:
from pystringbio.api.schemas import RefStringPpiTree
from pyrediscore.redantic import KeyStoreError, StoreKeyNotFound



store.load_model(RefStringPpiTree, 'root')
"""
Inject necessary values to store interaction data
"""
def store_interaction_data(interaction_data_iter):
    for i_datum in interaction_data_iter:
        for prot_root, prot_target in [ (i_datum.protein1, i_datum.protein2), (i_datum.protein2,i_datum.protein1) ]:
            i_obj = RefStringPpiTree(root=prot_root, data=[ {"target" : prot_target, "ppid" :  i_datum.ppid } ] )
        try :
            store.add(i_obj)
        except KeyStoreError:
            prev = store.get(i_obj.root, RefStringPpiTree)
            #print(prev)
            prev.data += i_obj.data
            #print(prev)
            #store.add(prev, replace=True)
store_interaction_data(i_data[:25])

[_ for _ in store.list_key(model=RefStringPpiTree) ]

In [56]:
[_ for _ in store.list_key(model=RefStringPpiTree) ]

['RefStringPpiTree:511145.b4403',
 'RefStringPpiTree:511145.b0943',
 'RefStringPpiTree:511145.b0005',
 'RefStringPpiTree:511145.b1002',
 'RefStringPpiTree:511145.b2023',
 'RefStringPpiTree:511145.b2126',
 'RefStringPpiTree:511145.b3646',
 'RefStringPpiTree:511145.b3766',
 'RefStringPpiTree:511145.b2979',
 'RefStringPpiTree:511145.b4558',
 'RefStringPpiTree:511145.b3498',
 'RefStringPpiTree:511145.b0004',
 'RefStringPpiTree:511145.b1006',
 'RefStringPpiTree:511145.b0946',
 'RefStringPpiTree:511145.b0001',
 'RefStringPpiTree:511145.b0861',
 'RefStringPpiTree:511145.b3107',
 'RefStringPpiTree:511145.b0810',
 'RefStringPpiTree:511145.b0443',
 'RefStringPpiTree:511145.b0942',
 'RefStringPpiTree:511145.b2483',
 'RefStringPpiTree:511145.b0003',
 'RefStringPpiTree:511145.b0981',
 'RefStringPpiTree:511145.b3769',
 'RefStringPpiTree:511145.b2682',
 'RefStringPpiTree:511145.b0075']

In [64]:
from pystringbio.api.schemas import StringAC, UniprotAC
from typing import Union

StringTuple = Tuple[StringAC, StringAC]
UniprotTuple = Tuple[UniprotAC, UniprotAC]
def get_pairwise(string_tuple:Union[StringTuple, UniprotTuple]):
    pass

In [58]:
store.load_model(StringToUniprot, 'fr')
store.load_model(UniprotToString, 'fr')
store.load_model(RefStringPpiTree, 'root')
# read from

a_string_id = i_data[0].protein1
#
maper = store.get(a_string_id, model=StringToUniprot)
# StoreKeyNotFound
#print(maper)
a_uniprot_id = maper.to[0]

# We start here, let's get all its uniprot partners

print("I start from ", a_uniprot_id)
#
string_tr = store.get(a_uniprot_id, UniprotToString)
#StoreKeyNotFound
for string_id in string_tr.to:
    print("Translates into ", string_id)
    ppi_match = store.get(string_id, RefStringPpiTree)
  #  print(ppi_match)
    for ppi_node in ppi_match.data:
        #print( ppi_node.target, type(ppi_node.target).__name__ )
        uniprot_match = store.get(ppi_node.target, StringToUniprot)
        print(f"\t= interacts w/ [{ppi_node.target}]{uniprot_match}")
        #uniprot_partner = store.get(ppi_node.target)
        


I start from  P0AD86
Translates into  511145.b0001
	= interacts w/ [511145.b0810]fr='511145.b0810' to=['P0AEQ6']
	= interacts w/ [511145.b0946]fr='511145.b0946' to=['P75862']
	= interacts w/ [511145.b4403]fr='511145.b4403' to=['P37005']
	= interacts w/ [511145.b2126]fr='511145.b2126' to=['P0AD14']
	= interacts w/ [511145.b3769]fr='511145.b3769' to=['P00892', 'C1P619']
	= interacts w/ [511145.b0943]fr='511145.b0943' to=['P75860']
	= interacts w/ [511145.b0075]fr='511145.b0075' to=['P0AD79']
	= interacts w/ [511145.b1002]fr='511145.b1002' to=['P19926']
	= interacts w/ [511145.b2023]fr='511145.b2023' to=['P60595']
	= interacts w/ [511145.b3646]fr='511145.b3646' to=['P0AGM2']
	= interacts w/ [511145.b0861]fr='511145.b0861' to=['P0AE30']
	= interacts w/ [511145.b2682]fr='511145.b2682' to=['P76628']
	= interacts w/ [511145.b3107]fr='511145.b3107' to=['P42625']
	= interacts w/ [511145.b2979]fr='511145.b2979' to=['P0AEP9']
	= interacts w/ [511145.b0004]fr='511145.b0004' to=['P00934']
	= intera

In [10]:
post("http://127.0.0.1:8000/interactions/load/", json= [   i_datum.dict() for i_datum in i_data[:10] ] )

<Response [200]>

In [42]:
import re
r = re.compile(
    r'^[a-zA-Z0-9_\.]+$'
)
try:
    assert r.fullmatch("511145.b3766@")
except Exception:
    print("toto")

toto


In [2]:
from pydantic import BaseModel
from datetime import datetime
from typing import List, Optional

In [3]:
class User(BaseModel):
    id: int
    name = 'John Doe'
    signup_ts: Optional[datetime] = None
    friends: List[int] = []
        
class UserX(BaseModel):
    id: str
    name = 'John Doe'
    signup_ts: Optional[datetime] = None
    friends: List[int] = []

print(dir(User))
User.__fields__


['Config', '__abstractmethods__', '__annotations__', '__class__', '__class_vars__', '__config__', '__custom_root_type__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__exclude_fields__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_validators__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__include_fields__', '__init__', '__init_subclass__', '__iter__', '__json_encoder__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__post_root_validators__', '__pre_root_validators__', '__pretty__', '__private_attributes__', '__reduce__', '__reduce_ex__', '__repr__', '__repr_args__', '__repr_name__', '__repr_str__', '__schema_cache__', '__setattr__', '__setstate__', '__signature__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__try_update_forward_refs__', '__validators__', '_abc_impl', '_calculate_keys', '_copy_and_set_values', '_decompose_class', '_enforce_dict_if_root', '_get_value', '_init_private_attributes', '_iter', 'con

{'id': ModelField(name='id', type=int, required=True),
 'signup_ts': ModelField(name='signup_ts', type=Optional[datetime], required=False, default=None),
 'friends': ModelField(name='friends', type=List[int], required=False, default=[]),
 'name': ModelField(name='name', type=str, required=False, default='John Doe')}

In [8]:
user = User(id=123)
isinstance(user, BaseModel)

True

In [5]:
user = User(id=123)

type(user).__name__

'User'

In [2]:
from pyrediscore.redantic import RedisStore
store = RedisStore("127.0.0.1", 6379)
from pydantic import BaseModel
from datetime import datetime
from typing import List, Optional

class User(BaseModel):
    id: int
    name = 'John Doe'
    signup_ts: Optional[datetime] = None
    friends: List[int] = []
        


#@redis_store.store_by_model(User, "id", _prefix="altUser")
#def add_smt(uid):
#     return User(id=uid)

#@redis_store.get_by_model(User, _prefix="altUser")
#def get_smt(uid):
#    return uid

In [4]:
store.load_model(User, 'id')
store.add_many( [ User(id=i) for i in range(0,2000)], skip_check=True )

2000

In [3]:
store.load_model(User, 'id')
store.get("222")

b'{"id": 222, "signup_ts": null, "friends": [], "name": "John Doe"}'


User(id=222, signup_ts=None, friends=[], name='John Doe')

In [3]:
for u in store.list_key(model=User):
    print(u)

User:1577
User:1824
User:809
User:482
User:379
User:1152
User:980
User:1846
User:620
User:1657
User:263
User:1628
User:381
User:388
User:566
User:1515
User:906
User:1181
User:1591
User:1980
User:1649
User:1758
User:1063
User:376
User:234
User:159
User:1371
User:2
User:1735
User:575
User:1693
User:719
User:803
User:426
User:919
User:1393
User:187
User:487
User:1831
User:164
User:535
User:1681
User:1474
User:664
User:1350
User:1203
User:266
User:1261
User:1244
User:248
User:1685
User:800
User:232
User:605
User:303
User:1485
User:830
User:766
User:1697
User:477
User:547
User:16
User:1398
User:1959
User:1137
User:959
User:87
User:737
User:971
User:616
User:1335
User:405
User:1470
User:1276
User:1293
User:1263
User:1130
User:614
User:672
User:1762
User:190
User:1867
User:1114
User:1894
User:1046
User:237
User:927
User:467
User:1027
User:1384
User:1121
User:1903
User:265
User:658
User:880
User:1924
User:1067
User:309
User:98
User:943
User:1806
User:1880
User:1833
User:900
User:558
User:610
U

User:1549
User:939
User:792
User:988
User:993
User:720
User:1328
User:450
User:1331
User:245
User:1535
User:419
User:1540
User:1918
User:1622
User:410
User:1875
User:945
User:1751
User:352
User:882
User:1180
User:1654
User:14
User:1374
User:958
User:90
User:1807
User:1467
User:1048
User:247
User:1190
User:888
User:411
User:1606
User:912
User:177
User:944
User:885
User:1421
User:119
User:1920
User:1417
User:137
User:1544
User:767
User:1519
User:101
User:1195
User:619
User:623
User:89
User:961
User:1825
User:647
User:448
User:1938
User:1821
User:1871
User:1091
User:1145
User:315
User:843
User:544
User:1007
User:1860
User:1053
User:1847
User:1723
User:1677
User:622
User:421
User:1466
User:511
User:871
User:657
User:1085
User:1306
User:1744
User:1191
User:1317
User:1268
User:386
User:131
User:1509
User:541
User:1826
User:1101
User:679
User:1385
User:306
User:152
User:120
User:1302
User:1580
User:54
User:823
User:1651
User:1553
User:1843
User:1547
User:807
User:1103
User:1734
User:294
User:

In [3]:
store.load_model(User, 'id')
store.mget(["1531","781", "toto"], model = User)

[User(id=1531, signup_ts=None, friends=[], name='John Doe'),
 User(id=781, signup_ts=None, friends=[], name='John Doe'),
 None]

In [4]:
from pystringbio.pyredis_dantic import RedisStore
redis_store = RedisStore("127.0.0.1", 6379)

@redis_store.store_by_model(User, "id")
def add_smt(uid):
     return User(id=uid)
        

TypeError: store_by_model() missing 1 required positional argument: 'prefix'

In [42]:
User.__name__

'User'

In [7]:
len({"a": 10, 'A':2})

2

In [9]:
import re
s = "AA:LL"
re.sub(r'^([^:]+:)', '', s)


'LL'